# Image Classification Neural Network
For our project, we will make use of a model for image recognition in order to classify the content of our artworks

In [2]:
import pandas as pd
import requests
df = pd.read_csv("SothebysData_clean.csv")

In [3]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications import VGG16

In [4]:
df

,Unnamed: 0,Artist,Title,Location,Currency,Date,Signed,Sold,Selling Price,low_estimate,high_estimate,Category,Auction Name,Image
0,0,"French School, 18th Century, A white greyhound","Ecole française du XVIIIe siècle, Lévrier blanc",Paris,USD,2022,False,False,0.00,5550.50,7770.70,Old Masters,ANIMALS,https://sothebys-md.brightspotcdn.com/dims4/de...
1,1,François Léon Prieur-Bardin,On the Bosphorus,London,USD,2022,True,True,62830.65,39367.57,52490.10,Old Masters,The Orientalist Sale,https://sothebys-md.brightspotcdn.com/dims4/de...
2,2,Stefano Ussi,A Moroccan Guard,London,USD,2022,False,True,7936.50,5249.01,7873.51,Old Masters,The Orientalist Sale,https://sothebys-md.brightspotcdn.com/dims4/de...
3,3,Ludwig Deutsch,Before the Mosque,London,USD,2022,True,False,0.00,262450.50,393675.75,Old Masters,The Orientalist Sale,https://sothebys-md.brightspotcdn.com/dims4/de...
4,4,Eugène Girardet,The Caravan,London,USD,2022,True,False,0.00,65612.62,91857.67,Old Masters,The Orientalist Sale,https://sothebys-md.brightspotcdn.com/dims4/de...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1588,1588,Gustave Moreau,Femme sur un griffon,Paris,USD,2021,True,True,497435.81,0.00,0.00,19th Century Paintings,Art Impressionniste et Moderne Evening Sale,https://sothebys-md.brightspotcdn.com/dims4/de...
1589,1589,Marguerite Burnat-Provins,Portrait of a man,Paris,USD,2021,True,True,3077.20,0.00,0.00,19th Century Paintings,"Collection Pierre Le-Tan, Session I",https://sothebys-md.brightspotcdn.com/dims4/de...
1590,1590,Henry Bishop,Portrait of Marcolesco,Paris,USD,2021,True,True,3496.82,0.00,0.00,19th Century Paintings,"Collection Pierre Le-Tan, Session I",https://sothebys-md.brightspotcdn.com/dims4/de...
1591,1591,Marcellin Gilbert Desboutin,Boy sleeping on a table,Paris,USD,2021,False,True,3636.69,0.00,0.00,19th Century Paintings,"Collection Pierre Le-Tan, Session I",https://sothebys-md.brightspotcdn.com/dims4/de...


## First we need to create a function to download the images from the url in our dataset

In [5]:
def download_jpgs(dataframe):
    """ """
    #for cur, price, low_est, high_est in zip(ArtInfo["Currency"], ArtInfo["price"]
    for title, image  in zip(dataframe['Title'], dataframe['Image']):
        response = requests.get(image)
        #
        file = open(title[:200] +".png", "wb")
        file.write(response.content)
        file.close()
    

## Load the model

In [6]:
# Loading VGG16 model
model = VGG16()

#summarize the model
#model architecture is summarized to ensure model was loaded correctly
model.summary()

2022-05-08 23:30:29.060448: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## Function to process and prepare image for classification

In [7]:
#load an image from file
#poto is loded and reshaped to 224x224 for the expected size of the model
def process_image(image):
    image = load_img(image, target_size=(224, 224))

    #convert the image pixels to a numpy array
    image = img_to_array(image)

    #reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

    #prepare the image for the VGG model
    image = preprocess_input(image)
    return image

## Classify Image

In [8]:
def classify_image(image):
    # predict the probability across all ouput classes
    yhat = model.predict(image)
    #convert the probabilities to class labels
    label = decode_predictions(yhat)
    label = label[0][0]
    #retrieve the most likely result, e.g. the highest probability
    
    print(label)
    #return the classifification
    print('%s (%.2f%%)' % (label[1], label[2]*100))

## Testing the model on a few selected images


In [9]:
image = process_image("Greyhound.jpg")
classify_image(image)

2022-05-08 23:30:35.347874: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


('n02091244', 'Ibizan_hound', 0.6988309)
Ibizan_hound (69.88%)


In [10]:
image = process_image("SandroBotticelli.jpg")
classify_image(image)

('n06359193', 'web_site', 0.45834818)
web_site (45.83%)


In [11]:
image = process_image("Number19.jpg")
classify_image(image)

('n01687978', 'agama', 0.07419421)
agama (7.42%)
